# CSE 153 Assignment 2

## Task 2: Harmonization

For this task, we will create a model that can generate harmonies and a backing track when provided a melody. (PROVIDE MORE DETAILS)

In [ ]:
# !apt install fluidsynth
# !git clone https://github.com/jthickstun/anticipation.git
# !pip install ./anticipation
# !pip install -r anticipation/requirements.txt

In [10]:
# IMPORTS
import pickle
import os
import sys
import miditoolkit
import numpy as np

In [ ]:
# anticipation
import sys,time

import midi2audio
import transformers
from transformers import AutoModelForCausalLM

from IPython.display import Audio

from anticipation import ops
from anticipation.sample import generate
from anticipation.tokenize import extract_instruments
from anticipation.convert import events_to_midi,midi_to_events
from anticipation.visuals import visualize
from anticipation.config import *
from anticipation.vocab import *

We will use the [POP909 dataset](https://arxiv.org/abs/2008.07142), which contains 909 piano arrangements of popular songs. Each song is broken into 3 midi files (MELODY, BRIDGE (for the harmonies), and PIANO (for the accompaniment)). The dataset comes with information about beat, chords, and key changes. We will be using their provided data preprocessing scripts to access these midi files.

In [17]:
# Pre-processing script by Ziyu Wang, Ke Chen, Junyan Jiang, Yiyi Zhang, Maoran Xu, 
# Shuqi Dai, Guxian Bin, and Gus Xia (POP909 team)

def preprocess_midi(path):
    midi_obj = miditoolkit.midi.parser.MidiFile(path)

    melody = list(sorted(midi_obj.instruments[0].notes, key=lambda x: x.start))
    # unsorted_melody = midi_obj.instruments[0].notes
    # for n, note in enumerate(melody):
    #     if note != unsorted_melody[n]:
    #         print(note, unsorted_melody[n])
    bridge = list(sorted(midi_obj.instruments[1].notes, key=lambda x: x.start))
    piano = list(sorted(midi_obj.instruments[2].notes, key=lambda x: x.start))

    data = {
        "melody": melody,
        "ornamentation": bridge,
        "harmony": piano
    }
    
    return data

def preprocess_pop909(midi_root, save_dir):
    save_py = []
    midi_paths = [f"{d}/{d}.mid" for d in os.listdir(midi_root)]
    i = 0
    
    for path in midi_paths:
        print(' ', end='[{}]'.format(path), flush=True)
        filename = midi_root + path
        try:
            data = preprocess_midi(filename)
            # print(data["melody"])
        except KeyboardInterrupt:
            print(' Abort')
            return
        except EOFError:
            print('EOF Error')
            return
        break
    #     save_py.append(data)
    # save_py = np.array(save_py)
    # print(save_py.size)
    # np.save("pop909-event-token.npy", save_py)
            
    
# replace the folder with your POP909 data folder
preprocess_pop909("POP909/","midi_data/")

 [038/038.mid]

In [ ]:
SMALL_MODEL = 'stanford-crfm/music-small-800k'     # faster inference, worse sample quality

# load an anticipatory music transformer
model = AutoModelForCausalLM.from_pretrained(SMALL_MODEL).cuda()

# a MIDI synthesizer
fs = midi2audio.FluidSynth('/usr/share/sounds/sf2/FluidR3_GM.sf2')

# the MIDI synthesis script
def synthesize(fs, tokens):
    mid = events_to_midi(tokens)
    mid.save('tmp.mid')
    fs.midi_to_audio('tmp.mid', 'tmp.wav')
    return 'tmp.wav'

In [ ]:
# load in midi, synthesizes the first 30 seconds of the loaded MIDI file by clipping the loaded sequence of events.
# The ops.clip function takes a list of events, a start_time, and an end_time, and returns the events in the interval 
# [start_time,end_time].

events = midi_to_events('anticipation/examples/strawberry.mid') # EDIT FILE TO CORRECT MIDI
Audio(synthesize(fs, ops.clip(events, 0, 30)))

@inproceedings{pop909-ismir2020,
    author = {Ziyu Wang* and Ke Chen* and Junyan Jiang and Yiyi Zhang and Maoran Xu and Shuqi Dai and Guxian Bin and Gus Xia},
    title = {POP909: A Pop-song Dataset for Music Arrangement Generation},
    booktitle = {Proceedings of 21st International Conference on Music Information Retrieval, {ISMIR}},
    year = {2020}
}